In [13]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from sklearn import neighbors
from sklearn.metrics import roc_curve, auc

In [14]:
from simulate import simulate_samples
from plots import plot_simulation, plot_supervised, plot_cluster_with_normal
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Define default values for sliders
default_values = {
    'n_cells': 10000,
    'frac_leukemic_AML1': 0.1,
    'frac_leukemic_AML2': 0.01,
    'frac_leukemic_AML3': 0.1,
    'mean_healthy_x': -1,
    'std_healthy_x': 0.5,
    'mean_healthy_y': -1,
    'std_healthy_y': 0.5,
    'dist_healthy_LAIP1': 3,
    'std_LAIP1_x': 0.5,
    'std_LAIP1_y': 0.5,
    'dist_healthy_LAIP2': 3,
    'std_LAIP2_x': 0.5,
    'std_LAIP2_y': 0.5
}

# Create sliders with default values
slider_specs = {
    'n_cells': ('Total cells', 1, 100000, 1),
    'frac_leukemic_AML1': ('Fraction leukemic cells in AML1', 0, 1, 0.01),
    'frac_leukemic_AML2': ('Fraction leukemic cells in AML2', 0, 1, 0.01),
    'frac_leukemic_AML3': ('Fraction leukemic cells in AML3', 0, 1, 0.01),
    'mean_healthy_x': ('Mean of non-leukemic cluster (X-axis)', -10, 10, 0.1),
    'std_healthy_x': ('Standard deviation of non-leukemic cluster (X-axis)', 0, 3, 0.1),
    'mean_healthy_y': ('Mean of non-leukemic cluster (Y-axis)', -10, 10, 0.1),
    'std_healthy_y': ('Standard deviation of non-leukemic cluster (Y-axis)', 0, 3, 0.1),
    'dist_healthy_LAIP1': ('Distance NBM/LAIP1', 0, 4, 0.01),
    'std_LAIP1_x': ('Standard deviation of LAIP1+ cluster (X-axis)', 0, 3, 0.1),
    'std_LAIP1_y': ('Standard deviation of LAIP1+ cluster (Y-axis)', 0, 3, 0.1),
    'dist_healthy_LAIP2': ('Distance between NBM/LAIP2', 0, 4, 0.01),
    'std_LAIP2_x': ('Standard deviation of LAIP2+ cluster (X-axis)', 0, 3, 0.1),
    'std_LAIP2_y': ('Standard deviation of LAIP2+ cluster (Y-axis)', 0, 3, 0.1)
}

slider_layout = widgets.Layout(width='400px')
sliders = {}

for key, (description, min_val, max_val, step) in slider_specs.items():
    if 'n_cells' in key:
        slider = widgets.IntSlider(value=default_values[key], min=min_val, max=max_val, step=step, description=description, style={'description_width': 'initial'}, layout=slider_layout)
    else:
        slider = widgets.FloatSlider(value=default_values[key], min=min_val, max=max_val, step=step, description=description, style={'description_width': 'initial'}, layout=slider_layout)
    sliders[key] = slider

# Define a function to restore default values
def restore_defaults(button):
    for key, slider in sliders.items():
        slider.value = default_values[key]

# Create a button to restore defaults
restore_defaults_button = widgets.Button(description="Restore Defaults", button_style='primary')
restore_defaults_button.on_click(restore_defaults)

# Define the interactive plot with sliders
simulation_plot = interactive(
    plot_simulation,
    **{key: slider for key, slider in sliders.items()}
)

supervised_plot = interactive(
    plot_supervised,
    **{key: slider for key, slider in sliders.items()}
)

cluster_with_normal_plot = interactive(
    plot_cluster_with_normal,
    **{key: slider for key, slider in sliders.items()}
)

In [16]:
# Display the interactive plot and the restore defaults button
display(simulation_plot, restore_defaults_button)

interactive(children=(IntSlider(value=10000, description='Total cells', layout=Layout(width='400px'), max=1000…

Button(button_style='primary', description='Restore Defaults', style=ButtonStyle())

In [5]:
# Display the interactive plot and the restore defaults button
display(supervised_plot, restore_defaults_button)

interactive(children=(IntSlider(value=10000, description='Total cells', layout=Layout(width='400px'), max=1000…

Button(button_style='primary', description='Restore Defaults', style=ButtonStyle())

In [6]:
display(cluster_with_normal_plot, restore_defaults_button)

interactive(children=(IntSlider(value=10000, description='Total cells', layout=Layout(width='400px'), max=1000…

Button(button_style='primary', description='Restore Defaults', style=ButtonStyle())